In [83]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score

Welcome to the year 2912, where your data science skills are needed to solve a cosmic mystery. We've received a transmission from four lightyears away and things aren't looking good.

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!


To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.

![title](img/spaceship.jpg)

In [50]:
datatrain=pd.read_csv('train.csv')#загружаем тренировочный датасет

In [51]:
datatrain=datatrain.dropna()#удаляем пустые строки

In [52]:
datatrain

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [53]:
def prepareData(datat,mode='train'):
    datat.dropna()
    #обрабатываем в первую очередь столбцы со специфическими данными(cabin,id)(ПЛОХАЯ ИДЕЯ,ТАК НЕ НАДО БЫЛО ДЕЛАТЬ)
    #data_group=datat['PassengerId'].str.split('_',expand=True)
    #data_group=data_group.rename({0: 'group', 1: 'id'}, axis='columns')
    #data_group=pd.get_dummies(data_group.drop(['id'],axis=1))
    
    #распарсим строку каюты ,удалим номер, а сторону и палубу выведем в отдельные признаки 
    data_cabin=datat['Cabin'].str.split('/',expand=True)
    data_cabin=data_cabin.rename({0: 'deck', 1: 'num',2:'side'}, axis='columns')
    data_cabin=data_cabin.drop(['num'],axis=1)
    data_cabin=pd.get_dummies(data_cabin)
    
    #теперь присвоим значения 1 и 0 целевой переменной
    data_target=pd.get_dummies(datat['Transported'])
    datat=datat.drop('Transported',axis=1)
    
    #убираем вышеупомянутые столбцы,создаем дамми переменные и нормализуем остальные
    datat=datat.drop(['PassengerId','Name','Cabin'],axis=1)
    datat=pd.get_dummies(datat)
    df_norm = (datat-datat.min())/(datat.max()-datat.min())
    
    #теперь осталось соединить все воедино
    data=pd.concat((df_norm, data_cabin), 1)
    #data=pd.concat((data, data_group), 1)
    dataFull=[]
    y=[]
    dataFull=pd.concat((data, data_target), 1)
    X=data.values
    y=data_target.values
    return X,y,dataFull

In [54]:
def print_logisitc_metrics(y_true, y_pred):#функция для вывода метрик классификации
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f'acc = {acc:.2f} F1-score = {f1:.2f}')

In [55]:
df=prepareData(datatrain)

C:\Users\sevas\AppData\Local\Temp\ipykernel_5740\2127565736.py:24: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  data=pd.concat((df_norm, data_cabin), 1)
C:\Users\sevas\AppData\Local\Temp\ipykernel_5740\2127565736.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  dataFull=pd.concat((data, data_target), 1)


In [56]:
len(df[2].columns)

28

In [57]:
X=df[0]
y=df[1]

In [58]:
y

array([[1, 0],
       [0, 1],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [59]:
len(X[0])

26

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
len(X_test[0])

26

In [79]:


'''inputs = keras.Input(shape=(5041,), name='input')
x = layers.Dense(528, activation='relu')(inputs)
x = layers.Dense(128, activation='relu')(x)
x=layers.dropout()
outputs = layers.Dense(2, activation='softmax')(x)'''
model = keras.Sequential([
    layers.Input(26,),
    layers.Dense(50, activation='relu'),
    layers.Dense(40, activation='relu'),
    layers.Dense(2, activation='softmax')
])

#model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

print(model.summary())# вывод структуры НС в консоль
model.fit(X, y, batch_size=32, epochs=15, validation_split=0.2)

training_result=model.evaluate(X_test, y_test)#оцениваем точность модели 
print(f"Точность:{training_result[1]} \n Функция потерь:{training_result[0]}")

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 50)                1350      
                                                                 
 dense_31 (Dense)            (None, 40)                2040      
                                                                 
 dense_32 (Dense)            (None, 2)                 82        
                                                                 
Total params: 3,472
Trainable params: 3,472
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/15
166/166 [==============================] - 1s 1ms/step - loss: 0.5675 - accuracy: 0.7021 - val_loss: 0.5242 - val_accuracy: 0.7239
Epoch 2/15
166/166 [==============================] - 0s 771us/step - loss: 0.5065 - accuracy: 0.7475 - val_loss: 0.4909 - val_accuracy: 0.7504
Epoch 3/15
166/166 [====

In [62]:
res = model.predict(X)
y_pred=np.argmax(res, axis=1)


207/207 [==============================] - 0s 487us/step


Теперь приступим к тестовому датасету

In [63]:
datat=pd.read_csv('test.csv')

In [64]:
pd.concat((datatrain,datat),0)

C:\Users\sevas\AppData\Local\Temp\ipykernel_5740\2240967151.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  pd.concat((datatrain,datat),0)
C:\Users\sevas\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\lib\pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
C:\Users\sevas\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\formatters.py:342: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,NaN
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,NaN
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,NaN
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,NaN


In [65]:
datat['Transported'] = datat['VIP']

In [66]:
datat

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,False
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,False
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,False
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,False
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,False
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,False
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,False
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,False


In [67]:
df=prepareData(datat)

C:\Users\sevas\AppData\Local\Temp\ipykernel_5740\2127565736.py:24: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  data=pd.concat((df_norm, data_cabin), 1)
C:\Users\sevas\AppData\Local\Temp\ipykernel_5740\2127565736.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  dataFull=pd.concat((data, data_target), 1)


In [68]:
datat=df[2]

In [69]:
len(X[0])

26

In [70]:
res = model.predict(X)
y_pred=np.argmax(res, axis=1)

207/207 [==============================] - 0s 488us/step


In [71]:
res=pd.DataFrame({'Transported':y_pred.astype(bool)})

In [72]:
submit=pd.read_csv('sample_submission.csv')

In [73]:
submit=submit.drop(['Transported'],axis=1)

In [74]:
pd.concat((submit,res),1)

C:\Users\sevas\AppData\Local\Temp\ipykernel_5740\2974881411.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  pd.concat((submit,res),1)


,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
6601,NaN,True
6602,NaN,True
6603,NaN,True
6604,NaN,False


In [75]:
len(datat.columns)

28

In [76]:
len(datatrain.columns)

14

In [77]:
datat

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,deck_C,deck_D,deck_E,deck_F,deck_G,deck_T,side_P,side_S,False,True
0,0.341772,0.000000,0.000000,0.00000,0.000000,0.000000,1.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,1,1,0
1,0.240506,0.000000,0.000356,0.00000,0.142260,0.000000,1.0,0.0,0.0,1.0,...,0,0,0,1,0,0,0,1,1,0
2,0.392405,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,1.0,0.0,0.0,...,1,0,0,0,0,0,0,1,1,0
3,0.481013,0.000000,0.263206,0.00000,0.009121,0.026266,0.0,1.0,0.0,1.0,...,1,0,0,0,0,0,0,1,1,0
4,0.253165,0.000865,0.000000,0.07658,0.000000,0.000000,1.0,0.0,0.0,1.0,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.430380,0.000000,0.000000,0.00000,0.000000,0.000000,1.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,1,1,0
4273,0.531646,0.000000,0.033514,0.00205,0.000504,0.006466,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
4274,NaN,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,1.0,0.0,...,0,1,0,0,0,0,1,0,1,0
4275,NaN,0.000000,0.106042,0.00000,0.000000,0.023482,0.0,1.0,0.0,1.0,...,0,1,0,0,0,0,1,0,1,0


In [78]:
datatrain

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False
